# **Datos de Game Spot**
***

## Librerias a ocupar
***

In [1]:
#Interactuar con el API
import requests
import json
from time import sleep

#procesamiento de datos
import numpy as np
import pandas as pd



## Datos obtenidos con la API de Game Spot
***

### Informacion y requisitos de la API
***

El API de Game Stop es de uso publico con la conción de no ocuparse con fines licrativos ni para competor con este.

Para poder ocuparla es necesarion crearse una cuenta en su sitio web, después dirigirse a la dirección del API, aceptar los terminos y condiciones y por ultimo crear tu Key para acceder al API.

A travez de esta uno puede acceder a distintos datos (Juegos, Entregas, Articulos, etcetera), en este caso nosotroa nos centraremos en las reseñas:

El URL del API para reseñas:  http://www.gamespot.com/api/reviews/?api_key=[Key personal]




### Interacción con la API
***

Primero veremos la función que implementará el *request()*

In [ ]:
def gamespotapi (llave: str, sesion: requests.session, offset: int):

    url='http://www.gamespot.com/api/reviews/?'

    parametros = {
        'api_key': llave,
        'format': 'json',       #Si no se especifica el request devuelde un HTML
        'offset': f'{offset}'   #Para poder hacer multiples requests
    }

    for _ in range(10):
        try:
            respuesta = sesion.get(f'{url}', headers= sesion.headers, params=parametros)
            respuesta.raise_for_status()
            return respuesta.json()
        
        except requests.exceptions.HTTPError as e:
            print("HTTPError:", e)
            sleep(6)

        except requests.exceptions.RequestException as e:
            print("Error de red:", e)            # Este fue el error más comun
            sleep(6)


Por otra parte está el codigo principal que se encarga de hacer los multiples requests y descargas:

In [ ]:
doc="Partes_separadas_del_proyecto/Alonso/llave.txt" # Aquí tengo guardada mi Key

with open(doc, mode="r", encoding="utf-8") as lista:
    llave = lista.readline().rstrip()

sesion= requests.session()

sesion.headers.update({'User-Agent': 'PUC Student proyect: https://github.com/claudiocanales721/Proyecto-cs-de-datos (aaqueveque@estudante.uc.cl)'})

#La documetación especificó que User-Agents comunes no serían haceptados
# así que preferí entregar la mayor cantidar de informacion sobre el request

offset=15800     #Esta es la variable que define que parte de los request tomar, va de 100 en 100
no_ultimo=True   #Para terminar el priceso cuando se halla legado al final
contador=159     #Me sirve para ver cuantos request llevo
while no_ultimo:

    respuesta=gamespotapi(llave,sesion,offset) #Se ocupa la funcion definida
    juegos=respuesta['results']                #esta es la información que nos interesa de la respuesta

    if len(juegos) < 100:             #Para terminar el priceso cuando se halla legado al final
        no_ultimo=False

    with open('data/gamespot_reseñas.json', 'w', encoding='utf-8') as f: #se crea o escribe en el documeto
        json.dump(juegos, f, indent=4)                                   #por cada descarga
    
    offset+=100       #Se actualiza el offset para no repetir descargas
    print(f'Se guardó el request N°{contador}, con {len(juegos)} juegos, en total hay {offset} juegos')
    contador+=1       #Para tener el feedback que todo va bien